In [1]:
import torch.utils.data as data
from PIL import Image
import os
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import matplotlib.pyplot as plt
from PIL import Image

def check_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

class Img_to_zero_center(object):
    def __int__(self):
        pass
    def __call__(self, t_img):
        '''
        :param img:tensor be 0-1
        :return:
        '''
        t_img=(t_img-0.5)*2
        return t_img

class Reverse_zero_center(object):
    def __init__(self):
        pass
    def __call__(self,t_img):
        t_img=t_img/2+0.5
        return t_img

In [3]:
class CACD(data.Dataset):
    def __init__(self, split='train', transforms=None, label_transforms=None):
        list_root = "/kaggle/input/facial-expression-recognition/lists"
        data_root = "/kaggle/input/facial-expression-recognition/FER"
        list_emotion = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        if split == "train":
            self.list_path= os.path.join(list_root,"train.txt")
        else:
            self.list_path = os.path.join(list_root, "test.txt")
        self.images_labels=[]#path
        self.transform=transforms
        self.label_transform=label_transforms
        with open(self.list_path) as fr:
            lines=fr.readlines()
            for line in lines:
                line.strip()
                item=line.split()
                image_label=[]
                image_label.append(os.path.join(data_root,list_emotion[int(item[1])],item[0]))
                try:
                    image_label.append(np.array(int(item[1])))
                except:
                  print(item[0])
                  print(item[1])
                self.images_labels.append(image_label)

    def __getitem__(self, idx):
        img_path,label=self.images_labels[idx]
        img=Image.open(img_path)

        if self.transform is not None:
            img=self.transform(img)
        if self.label_transform is None:
            label=torch.from_numpy(label)
        return img,label

    def __len__(self):
        return len(self.images_labels)


transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[1., 1., 1.]),
])

CACD_dataset=CACD("train", transforms, None)
train_loader = torch.utils.data.DataLoader(
    dataset=CACD_dataset,
    batch_size=32,
    shuffle=False
)
for idx,(img,label) in enumerate(train_loader):
    print(img.size())
    print(label.size())
    break

torch.Size([32, 3, 400, 400])
torch.Size([32])


In [4]:
import math
from torch.nn.parameter import Parameter
from torch.nn.functional import pad
from torch.nn.modules import Module
from torch.nn.modules.utils import _single, _pair, _triple

class _ConvNd(Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride,
                 padding, dilation, transposed, output_padding, groups, bias):
        super(_ConvNd, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        if transposed:
            self.weight = Parameter(torch.Tensor(
                in_channels, out_channels // groups, *kernel_size))
        else:
            self.weight = Parameter(torch.Tensor(
                out_channels, in_channels // groups, *kernel_size))
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def __repr__(self):
        s = ('{name}({in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        s += ')'
        return s.format(name=self.__class__.__name__, **self.__dict__)

class Conv2d(_ConvNd):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        super(Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, _pair(0), groups, bias)

    def forward(self, input):
        return _conv2d_same_padding(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

# custom conv2d, because pytorch don't have "padding='same'" option.
def _conv2d_same_padding(input, weight, bias=None, stride=(1,1), padding=1, dilation=(1,1), groups=1):

    input_rows = input.size(2)
    filter_rows = weight.size(2)
    effective_filter_size_rows = (filter_rows - 1) * dilation[0] + 1
    out_rows = (input_rows + stride[0] - 1) // stride[0]
    padding_needed = max(0, (out_rows - 1) * stride[0] + effective_filter_size_rows -
                  input_rows)
    padding_rows = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    rows_odd = (padding_rows % 2 != 0)
    padding_cols = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    cols_odd = (padding_rows % 2 != 0)

    if rows_odd or cols_odd:
        input = pad(input, [0, int(cols_odd), 0, int(rows_odd)])

    return F.conv2d(input, weight, bias, stride,
                  padding=(padding_rows // 2, padding_cols // 2),
                  dilation=dilation, groups=groups)

In [9]:
class AgeAlexNet(nn.Module):
    def __init__(self,pretrainded=False,modelpath=None):
        super(AgeAlexNet, self).__init__()
        assert pretrainded is False or modelpath is not None,"pretrain model need to be specified"
        self.features = nn.Sequential(
            Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(2,2e-5,0.75),

            Conv2d(96, 256, kernel_size=5, stride=1,groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(2, 2e-5, 0.75),

            Conv2d(256, 384, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),

            Conv2d(384, 384, kernel_size=3,stride=1,groups=2),
            nn.ReLU(inplace=True),

            Conv2d(384, 256, kernel_size=3,stride=1,groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.age_classifier=nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 8),
        )
        if pretrainded is True:
            self.load_pretrained_params(modelpath)

        self.Conv3_feature_module=nn.Sequential()
        self.Conv4_feature_module=nn.Sequential()
        self.Conv5_feature_module=nn.Sequential()
        self.Pool5_feature_module=nn.Sequential()
        for x in range(10):
            self.Conv3_feature_module.add_module(str(x), self.features[x])
        for x in range(10,12):
            self.Conv4_feature_module.add_module(str(x),self.features[x])
        for x in range(12,14):
            self.Conv5_feature_module.add_module(str(x),self.features[x])
        for x in range(14,15):
            self.Pool5_feature_module.add_module(str(x),self.features[x])


    def forward(self, x):
        self.conv3_feature=self.Conv3_feature_module(x)
        self.conv4_feature=self.Conv4_feature_module(self.conv3_feature)
        self.conv5_feature=self.Conv5_feature_module(self.conv4_feature)
        pool5_feature=self.Pool5_feature_module(self.conv5_feature)
        self.pool5_feature=pool5_feature
        flattened = pool5_feature.view(pool5_feature.size(0), -1)
        age_logit = self.age_classifier(flattened)
        return age_logit

    def load_pretrained_params(self,path):
        # step1: load pretrained model
        pretrained_dict = torch.load(path)
        # step2: get model state_dict
        model_dict = self.state_dict()
        # step3: remove pretrained_dict params which is not in model_dict
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # step4: update model_dict using pretrained_dict
        model_dict.update(pretrained_dict)
        # step5: update model using model_dict
        self.load_state_dict(model_dict)


class AgeClassify:
    def __init__(self, modelz = AgeAlexNet(pretrainded=False).cuda()):
        #step 1:define model
        self.model=modelz
        #step 2:define optimizer
        self.optim=torch.optim.Adam(self.model.parameters(),lr=1e-4,betas=(0.5, 0.999))
        #step 3:define loss
        self.criterion=nn.CrossEntropyLoss().cuda()

    def train(self,input,label):
        self.model.train()
        output=self.model(input)
        self.loss=self.criterion(output,label)

    def val(self,input):
        self.model.eval()
        output=F.softmax(self.model(input),dim=1).max(1)[1]
        return output

    def save_model(self,dir,filename):
        torch.save(self.model.state_dict(),os.path.join(dir,filename))

batch_size 512 20 epochs

In [8]:
from tqdm import tqdm
import os

#define
max_epoches = 20
batch_size = 256
save_interval = 20000
val_interval = 20000
saved_model_folder = '/kaggle/working/checkpoint'
check_dir(saved_model_folder)

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227, 227)),
    torchvision.transforms.ToTensor(),
    Img_to_zero_center()
])

train_dataset = CACD("train", transforms, None)
test_dataset = CACD("test", transforms, None)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

model=AgeClassify()
optim=model.optim

for epoch in range(max_epoches):
    for train_idx, (img,label) in enumerate(train_loader):
        img=img.cuda()
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()

        if (train_idx + 1)%50 == 0:
            print('step %d/%d, cls_loss = %.3f' % (train_idx, len(train_loader), loss))

        # save the parameters at the end of each save interval
        if train_idx*batch_size % save_interval == 0:
            model.save_model(dir=saved_model_folder,
                              filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')

        #val step

        if train_idx % val_interval == 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            print('val_acc = %.3f' % (train_correct.cpu().numpy()/train_total))

checkpoint has been created!


100%|██████████| 11/11 [00:16<00:00,  1.48s/it]


validate has been finished!
val_acc = 0.173
step 49/101, cls_loss = 2.067
step 99/101, cls_loss = 1.866
checkpoint has been created!


100%|██████████| 11/11 [00:17<00:00,  1.60s/it]


validate has been finished!
val_acc = 0.249
step 49/101, cls_loss = 1.779
step 99/101, cls_loss = 1.751
checkpoint has been created!


...

validate has been finished!
val_acc = 0.610
step 49/101, cls_loss = 0.552
step 99/101, cls_loss = 0.620
checkpoint has been created!


100%|██████████| 11/11 [00:17<00:00,  1.57s/it]


validate has been finished!
val_acc = 0.617
step 49/101, cls_loss = 0.518
step 99/101, cls_loss = 0.591
checkpoint has been created!


100%|██████████| 11/11 [00:17<00:00,  1.56s/it]


validate has been finished!
val_acc = 0.605
step 49/101, cls_loss = 0.582
step 99/101, cls_loss = 0.511


batch_size 64 epoch 20

In [11]:
modelz = AgeAlexNet(pretrainded=True,modelpath="/kaggle/working/checkpoint/epoch_19_iter_0.pth").cuda()
model=AgeClassify(modelz)
optim=torch.optim.Adam(modelz.parameters(),lr=1e-5,betas=(0.5, 0.999))

<ipython-input-9-bab3d8e81a13>:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(path)


In [14]:
from tqdm import tqdm
import os

#define
max_epoches = 10
batch_size = 64
save_interval = 400
val_interval = 400

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

saved_model_folder = '/kaggle/working/checkpoint2'
check_dir(saved_model_folder)

for epoch in range(max_epoches):
    for train_idx, (img,label) in enumerate(train_loader):
        img=img.cuda()
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()

        if train_idx % 36 == 0:
            print('step %d/%d, cls_loss = %.3f' % (train_idx, len(train_loader), loss))

        # save the parameters at the end of each save interval
        if (train_idx+1) % save_interval == 0:
            model.save_model(dir=saved_model_folder,
                              filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')
            print(epoch)

        #val step

        if (train_idx+1) % val_interval == 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            print('val_acc = %.3f' % (train_correct.cpu().numpy()/train_total))

step 0/403, cls_loss = 0.284
step 36/403, cls_loss = 0.458
step 72/403, cls_loss = 0.345
step 108/403, cls_loss = 0.401
step 144/403, cls_loss = 0.241
step 180/403, cls_loss = 0.220
step 216/403, cls_loss = 0.292
step 252/403, cls_loss = 0.341
step 288/403, cls_loss = 0.309
step 324/403, cls_loss = 0.309
step 360/403, cls_loss = 0.257
step 396/403, cls_loss = 0.247
checkpoint has been created!
0


100%|██████████| 44/44 [00:18<00:00,  2.41it/s]


validate has been finished!
val_acc = 0.637
step 0/403, cls_loss = 0.291
step 36/403, cls_loss = 0.383
step 72/403, cls_loss = 0.280
step 108/403, cls_loss = 0.241
step 144/403, cls_loss = 0.376
step 180/403, cls_loss = 0.323
step 216/403, cls_loss = 0.140
step 252/403, cls_loss = 0.275
step 288/403, cls_loss = 0.445
step 324/403, cls_loss = 0.476
step 360/403, cls_loss = 0.555
step 396/403, cls_loss = 0.359
checkpoint has been created!
1


100%|██████████| 44/44 [00:16<00:00,  2.64it/s]


validate has been finished!
val_acc = 0.625
step 0/403, cls_loss = 0.300
step 36/403, cls_loss = 0.319
step 72/403, cls_loss = 0.403
step 108/403, cls_loss = 0.157
step 144/403, cls_loss = 0.285
step 180/403, cls_loss = 0.260
step 216/403, cls_loss = 0.436
step 252/403, cls_loss = 0.330
step 288/403, cls_loss = 0.395
step 324/403, cls_loss = 0.253
step 360/403, cls_loss = 0.317
step 396/403, cls_loss = 0.286
checkpoint has been created!
2


...

validate has been finished!
val_acc = 0.623
step 0/403, cls_loss = 0.139
step 36/403, cls_loss = 0.106
step 72/403, cls_loss = 0.194
step 108/403, cls_loss = 0.197
step 144/403, cls_loss = 0.209
step 180/403, cls_loss = 0.346
step 216/403, cls_loss = 0.255
step 252/403, cls_loss = 0.500
step 288/403, cls_loss = 0.144
step 324/403, cls_loss = 0.175
step 360/403, cls_loss = 0.120
step 396/403, cls_loss = 0.280
checkpoint has been created!
8


100%|██████████| 44/44 [00:16<00:00,  2.63it/s]


validate has been finished!
val_acc = 0.620
step 0/403, cls_loss = 0.206
step 36/403, cls_loss = 0.215
step 72/403, cls_loss = 0.186
step 108/403, cls_loss = 0.141
step 144/403, cls_loss = 0.197
step 180/403, cls_loss = 0.250
step 216/403, cls_loss = 0.274
step 252/403, cls_loss = 0.232
step 288/403, cls_loss = 0.215
step 324/403, cls_loss = 0.185
step 360/403, cls_loss = 0.236
step 396/403, cls_loss = 0.240
checkpoint has been created!
9


100%|██████████| 44/44 [00:17<00:00,  2.59it/s]


validate has been finished!
val_acc = 0.618


In [19]:
!mkdir /kaggle/working/model
!cp /kaggle/working/checkpoint2/epoch_9_iter_399.pth /kaggle/working/model

In [20]:
!zip -r model.zip /kaggle/working/model

import kagglehub

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = '/kaggle/working/model.zip'

MODEL_SLUG = 'faceAlexnet_pretrain4emotion' # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = 'default' # Replace with variation slug.

kagglehub.model_upload(
  handle = f"poongln/{MODEL_SLUG}/keras/{VARIATION_SLUG}",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2024-12-30')

  adding: kaggle/working/model/ (stored 0%)
  adding: kaggle/working/model/epoch_9_iter_399.pth (deflated 7%)
Uploading Model https://www.kaggle.com/models/poongln/faceAlexnet_pretrain4emotion/keras/default ...
Model 'faceAlexnet_pretrain4emotion' does not exist or access is forbidden for user 'poongln'. Creating or handling Model...
Model 'faceAlexnet_pretrain4emotion' Created.
Starting upload for file /kaggle/working/model.zip


Uploading: 100%|██████████| 211M/211M [00:03<00:00, 68.8MB/s] 

Upload successful: /kaggle/working/model.zip (201MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/poongln/faceAlexnet_pretrain4emotion/keras/default
